In [1]:
import wolframalpha as wa
import telebot
import sympy as sp
import subprocess

In [2]:
def chech_internet():
    internet = False
    while not internet:
        try:
            subprocess.check_call(["ping", "-c 1", "www.google.ru"])
            internet = True
        except subprocess.CalledProcessError:
            internet = False
    return internet

In [3]:
def wolfram(question):
    #if not chech_internet():
    #    return 'This internet is shit!'
    app_id = 'ELQ8HA-KQKRHE2KVW'
    client = wa.Client(app_id)
    try:
        res = client.query(question)
        list = next(res.results)['subpod']
        answer = ''
        try:
            answer = list['img']['@alt']
        except:
            try:
                for i in list:
                    answer += i['img']['@alt'] + '\n'
            except:
                answer = 'Wolfram fucked up!'
    except:
        answer = 'You fucked up!'
    return answer

In [10]:
def parse_input(string):
    def is_any_of(_array, _x):
        for x in _array:
            if x == _x: return True
        return False
    integral_commands = ['integrate', 'integral']
    diff_commands = ['diff', 'deriv', 'differentiate', 'derivative']
    lim_commands = ['lim', 'limit']
    extend_commands = ['extend', 'expand']
    simplify_commands = ['simplify', 'simple']
    series_commands = ['series', 'taylor', 'macloren']
    solve_commands = ['solve']
    wolfram_commands = ['wolfram', 'wolframalpha']

    command = string[:string.find(' ')]
    args = string[string.find(' ') + 1:]
    result = ''
    if is_any_of(integral_commands, command):
        x = sp.symbols('x')
        lims_idx = args.find('from')
        try:
            if lims_idx == -1:
                x = sp.Symbol('x')
                result = sp.integrate(args, x)
            else:
                funct = args[:lims_idx - 1]
                lims = args[lims_idx:].split(' ')
                result = sp.integrate(funct, (x, lims[1], lims[3]))
                result = sp.integrate(funct, (x, lims[1], lims[3]))
        except:
            result = 'You fucking donkey!'
        
    elif is_any_of(diff_commands, command):
        try:
            var_idx = args.find('var')
            if var_idx == -1:
                x = sp.Symbol('x')
                result = sp.diff(args, x)
            else:
                _args = args[var_idx:].replace('=', ' ').split(' ')
                var = sp.Symbol(_args[-1])
                result = sp.diff(args[:var_idx - 1], var)
        except:
            result = 'You fucking donkey!'
    elif is_any_of(lim_commands, command):
        try:
            lim_idx = args.find('->') - 1
            while not str(args[lim_idx]).isalpha(): lim_idx -= 1
            while str(args[lim_idx]) != ' ': lim_idx -= 1
            _args = args[lim_idx + 1:].replace('->', ' ').split(' ')
            funct = args[:lim_idx]
            var = sp.Symbol(_args[0])
            result = sp.limit(funct, var, _args[-1])
        except:
            result = 'You fucking donkey!'
    elif is_any_of(extend_commands, command):
        try:
            result = sp.expand(args)
        except:
            result = 'You fucking donkey!'
    elif is_any_of(simplify_commands, command):
        try:
            result = sp.simplify(args)
        except:
            result = 'You fucking donkey!'
    elif is_any_of(series_commands, command):
        try:
            var_idx = args.find('var')
            if var_idx == -1:
                x = sp.Symbol('x')
                result = sp.series(args, x)
            else:
                _args = args[var_idx:].replace('=', ' ').split(' ')
                var = sp.Symbol(_args[-1])
                result = sp.series(args[:var_idx - 1], var)
        except:
            result = 'You fucking donkey!'
    elif is_any_of(solve_commands, command):
        try:
            var_idx = args.find('var')
            if var_idx == -1:
                x = sp.Symbol('x')
                result = sp.solve(args, x)
            else:
                _args = args[var_idx:].replace('=', ' ').replace(',', ' ').split(' ')
                arg_str = ''
                for _arg in _args[1:]:
                    if _arg.isalpha(): arg_str += _arg + ' '
                vars = sp.symbols(arg_str)
                result = str(sp.solve(args[:var_idx - 1].split(','), vars))
        except:
            result = 'You fucking donkey!'
    elif is_any_of(wolfram_commands, command):
        try:
            result = wolfram(args)
        except:
            result = 'You fucking donkey!'
    else:
        result = 'What the fuck are you going to say!?'
    return result

In [11]:
token = '5367239215:AAGTsWVilSMwJTlCBu-Ai_Iga9GjwpgsBxI'
#print(str(wiki.summary('euler equation')))
bot = telebot.TeleBot(token = token)
# Функция, обрабатывающая команду /start
@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Что считаем?')
# Получение сообщений от юзера
@bot.message_handler(content_types=["text"])
def handle_text(message):
    bot.send_message(message.chat.id, parse_input(message.text))
# Запускаем бота
bot.polling(none_stop=True, interval=0)